In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Collision resolution: Open addressing

In chaining, key-value pairs are stored in data structures external to the hash table array. Open addressing is the name for techniques that resolve collisions by searching for empty locations in the hash table array. When adding a key-value pair the basic idea is to start at the bucket that the key hashes to and then proceed to additional buckets in a *probe sequence* until an empty bucket is found. When searching for a key, the buckets are probed in the same sequence until the key is found or an empty bucket is found.

A drawback of open addressing is that the number of key-value pairs $n$ cannot exceed the hash table size $m$ and performance degrades severely as the hash table becomes almost full. Open addressing also places more requirements on the hash function as we shall see.

**Exercise 1** What is the maximum load factor when using open addressing?

The sequence of array locations probed depends on the key that is being used to access the hash table. The hash function is extended to have the form $h(k, i)$ where the second integer parameter $i$ takes on the values $0, 1, 2, ..., m-1$. The parameter $i$ is called the *probe number*. When we search for a key $k$ we start by calling the hash function with $i = 0$. If the bucket $h(k, 0)$ is occupied, then we increment $i$ so that $i=1$ and try again. If the bucket $h(k, 1)$ is occupied, then we increment $i$ so that $i=2$ and try again. We repeat the process of incrementing $i$ until we find an open bucket or until $i = m$. If $i = m$ then we have probed every location in the hash table without finding an open bucket and we conclude that the hash table array is full.

For open addressing to work correctly we need to ensure that the probe sequence

$\langle h(k, 0), h(k, 1), h(k, 2), ..., h(k, m-1) \rangle$

is a permutation of $\langle 0, 1, 2, ..., m-1 \rangle$ so that every table location is probed.

Adding a key-value pair to the hash table using open addressing can be performed with the algorithm `put` where `put` returns the previous value mapped to they key or `null` if no previous mapping for the key exists in the table:

```
put(key, value) {
    i = 0
    do {
        j = h(k, i)
        if t[j] == null || t[j] == DEL {
            set t[j] to the key-value pair
            return null
        }
        else if t[j].key == key {
            result = t[j].value
            set t[j].value to value
            return result
        }
        else {
            i = i + 1;
        }
    } while i < m
    error "hash table full"
}
```

Note that the hash table can now contain the special value `DEL` which is used to indicate that a bucket formerly held a key-value pair that was removed at some point. Why is the value `DEL` required in addition to the value `null`? Imagine that we have three distinct keys $k_1$, $k_2$, and $k_3$ where $h(k_1, i) = h(k_2, i) = h(k_3, i)$ for all values of $i$ and we insert the key-value pairs $(k_1, v_1)$, $(k_2, v_2)$, and $(k_3, v_3)$ into the hash table. Assume that the probe sequences for adding the key-value pairs are:

* $\langle h(k_1, 0) \rangle = \langle 3 \rangle$ 
* $\langle h(k_2, 0), h(k_2, 1) \rangle = \langle 3, 4 \rangle$ 
* $\langle h(k_3, 0), h(k_3, 1), h(k_3, 2) \rangle = \langle 3, 4, 5 \rangle$ 

Then a visualization of the hash-table is as follows:

![](../resources/images/hashing/del-1.png)

To delete $(k_2, v_2)$ we use the probe sequence $\langle h(k_2, 0), h(k_2, 1) \rangle = \langle 3, 4 \rangle$ to find $k_2$. If $(k_2, v_2)$ is deleted by replacing it with `null` then a visualization of the hash-table is as follows:

![](../resources/images/hashing/del-2.png)

If we search the above hash table for $k_3$ we probe with $i=0$ yielding the index $h(k_3, 0) = 3$. There is a key-value pair in $t[3]$ so we now probe with $i=1$ yielding the index $h(k_3, 1) = 4$. The value `null` is in $t[4]$ so we erroneously conclude that $k_3$ is not in the hash table.

Instead of using `null` to indicate the removal of a key-value pair we use a special value `DEL` as shown below.

![](../resources/images/hashing/del-3.png)

Now when we search for $k_3$ we see that $t[4]$ is `DEL` which indicates that the search should not stop yet. We probe with $i=2$ yielding the index $h(k_3, 2) = 5$. The  key-value pair in $t[5]$ has a key equal to $k_3$ so we conclude that the table contains the searched for key.

**Exercise 2** Give an algorithm for searching for a key $k$ in the hash table.

**Exercise 3** Give an algorithm for removing the key-value pair having the key $k$ from a hash table.

Open addressing schemes differ in how they define the probe sequence. Three common probe sequences are described in the following cells.

## Linear probing

Linear probing starts at the bucket that the key $k$ hashes to. If that bucket is full, then next bucket is examined. If that bucket is full, then the next bucket is examined until all of the buckets in the hash table have been examined.

Suppose that we have a helper or *auxillary* hash function $h'(k)$ that does not involve the probe number. Then linear probing using the hash function

$$h(k, i) = (h'(k) + i)\ \text{mod}\ m$$

The $\text{mod}\ m$ is required to make sure array indexes wrap around to 0 when $(h'(k) + i) > m$. Linear probing is the probe sequence used in the example in the previous cell.

In a full hash table, linear probing probes the array locations $t[j], t[j + 1], t[j+2], ..., t[m-1], t[0], t[1], ..., t[j - 1]$ where $j = h(k)$ is the hash value computed using the key $k$. It should be clear to the reader that linear probing is guaranteed to probe every table location.

Linear probing suffers from a problem called *primary clustering* where occupied array locations tend to cluster in long runs. The following figure helps to illustrate why runs of occupied array locations occur:

![](../resources/images/hashing/linear-probing.png)

Suppose that array locations 3, 4, and 5 (shown shaded gray) are occupied. Suppose that our hashing function $h(k)$ distributes keys evenly throughout the hash table. What is the probability that a random key will be inserted into location 10? The probability is equal to $1 / 11$ because every array location is equally likely. What is the probability that a random key will be inserted into location 6? The probability is equal to $4 / 11$ because a hash value of 3, 4, 5, or 6 will cause the key to be inserted into location 6. The proability of inserting at the end of a continuous run of occupied locations increases as the run becomes longer; thus, runs tend to get longer.

The Java class `java.util.IdentityHashMap` is a hash table implementation that uses linear probing.

## Quadratic probing

Quadratic probing uses the hash function

$$h(k, i) = (h'(k) + c_1 i + c_2 i^2)\ \text{mod}\ m$$

where $c_1$ and $c_2$ are real-valued constants and $c_2 \neq 0$.

Consider the following quadratic probing example where the hash table has $m = 2^4 = 16$ buckets, $c_1 = c_2 = 0.5$, and $h'(k) = 3$ for some key $k$. The values of the probe sequence are shown in the table below where $g(k, i) = (h'(k) + c_1 i + c_2 i^2)$ is the value of $h(k, i)$ before applying the modulo operation.

| i | g(k, i) | h(k, i) |
| :-: | :-: | :-: |
| 0 | 3 | 3 |
| 1 | 4 | 4 |
| 2 | 6 | 6 |
| 3 | 9 | 9 |
| 4 | 13 | 13 |
| 5 | 18 | 2 |
| 6 | 24 | 8 |
| 7 | 31 | 15 |
| 8 | 39 | 7 |
| 9 | 48 | 0 |
| 10 | 58 | 10 |
| 11 | 69 | 5 |
| 12 | 81 | 1 |
| 13 | 94 | 14 |
| 14 | 108 | 12 |
| 15 | 123 | 11 |

Notice that the probe sequence is a permutation of the values $0, 1, 2, ..., 15$ which means that every bucket in the hash table will be probed. If the table size is an integer power of $2$ and $c_1 = c_2 = 0.5$ then it can be proven that quadratic probing will probe every bucket in the hash table.

Now consider an example where the hash table has $m = 17$ buckets, $c_1 = c_2 = 0.5$, and $h'(k) = 3$ for some key $k$. The values of the probe sequence are shown in the table below

| i | g(k, i) | h(k, i) |
| :-: | :-: | :-: |
| 0 | 3 | 3 |
| 1 | 4 | 4 |
| 2 | 6 | 6 |
| 3 | 9 | 9 |
| 4 | 13 | 13 |
| 5 | 18 | 1 |
| 6 | 24 | 7 |
| 7 | 31 | 14 |
| 8 | 39 | 5 |
| 9 | 48 | 14 |
| 10 | 58 | 7 |
| 11 | 69 | 1 |
| 12 | 81 | 13 |
| 13 | 94 | 9 |
| 14 | 108 | 6 |
| 15 | 123 | 4 |
| 16 | 139 | 3 |

Notice that the hash values start to repeat after about $m / 2$ values are probed. When $m$ is a prime number many choices of $c_1$ and $c_2$ will cause the first $(m - 1) / 2$ hash values to be distinct but repeats may occur for any value of $i > (m - 1) / 2$. This means that insertions are not guaranteed to succeed if the load factor $\alpha > 0.5$.

**Exercise 4** Prove that for $c_1 = c_2 = 0.5$ the hash value $h(k, i)$ is always an integer.

**Exercise 5** If $m$ is not prime and not an integer power of 2 then quadratic probing can perform very badly. Compute the probe sequence for $m=12$, $c_1 = 0$, $c_2 = 1$, and $h'(k) = 8$.

Quadratic probing does not suffer from primary clustering but it does suffer from a milder form of clustering called *secondary clustering*. The problem occurs because $h'(k)$ determines the entire probe sequence. If two keys $k_1$ and $k_2$ result in $h'(k_1) = h'(k_2)$ then both keys will have identical probe sequences. This means that quadratic probing uses only $m$ different probe sequences for any given hash table (which is a tiny fraction of the $m!$ possible probe sequences).

## Double hashing

Double hashing uses two auxillary hash functions $h_1(k)$ and $h_2(k)$ such that the overall hash function is

$$h(k, i) = (h_1(k) + i h_2(k))\ \text{mod}\ m$$

Because $h_1$ and $h_2$ are evaluated only once during a search/add/remove operation the hashing function has the form

$$h(k, i) = (c_1 + i c_2)\ \text{mod}\ m$$

where $c_1$ and $c_2$ are constants that depend on the key $k$. The probing sequence starts at $c_1$ and then takes steps of size $c_2$ (mod $m$) through the hash table. It should be obvious to the reader that the step size $c_2 = h_2(k)$ determines what fraction of the buckets are probed.

**Exercise 6** If the hash table has an even number of buckets and $h_2(k) = 2$ for some key $k$ what fraction of the buckets will the probe sequence visit?

**Exercise 7** If the hash table has an odd number of buckets and $h_2(k) = 2$ for some key $k$ what fraction of the buckets will the probe sequence visit?

If the number of buckets $m$ and $h_2(k)$ have a greatest common divisor $d > 1$ then the probe sequence will not visit all of the buckets. 

**Exercise 8** Suppose $m = 12$ and $h_2(k) = 3$, then the greatest common divisor is $d = 3$. What fraction of the buckets are visited by the probe sequence?

**Exercise 9** Suppose $m = 12$ and $h_2(k) = 4$, then the greatest common divisor is $d = 4$. What fraction of the buckets are visited by the probe sequence?

**Exercise 10** Suppose $m = 12$ and $h_2(k) = 6$, then the greatest common divisor is $d = 6$. What fraction of the buckets are visited by the probe sequence?

For the probe sequence to visit every bucket, $m$ and $h_2(k)$ must have no common divisor except for $1$. We say that two integers are *relatively prime*, or *coprime*, if there is no integer greater than one that divides them both.

One way to guarantee that $m$ and $h_2(k)$ are coprime is to let $m$ be an integer power of $2$ and design $h_2(k)$ to always return an odd number.

Another way is to let $m$ be prime and design $h_2(k)$ to always return a positive integer less than $m$. For example, we can choose $h_2(k) = 1 + (k\ \text{mod}\ (m - 1))$.

Double hashing does not suffer from the clustering problems that linear and quadratic probing suffer from.